# Released under MIT License

Copyright (c) 2013 Mark Otto.

Copyright (c) 2017 Andrew Fong.

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

https://www.kaggle.com/datasets/borhanitrash/animal-image-classification-dataset

# About Dataset

***Dataset Summary***

The Animal Image Classification Dataset is a comprehensive collection of images tailored for the development and evaluation of machine learning models in the field of computer vision. It contains 3,000 JPG images, carefully segmented into three classes representing common pets and wildlife: cats, dogs, and snakes.

Dataset Contents
cats/: A set of 1,000 JPG images of cats, showcasing a wide array of breeds, environments, and postures.

dogs/: A diverse compilation of 1,000 dog images, capturing a multitude of breeds in various activities and settings.

snakes/: An assortment of 1,000 images of snakes, depicting numerous species in both natural and controlled habitats. Image Details:

Resolution: Each image maintains a uniform resolution of 256x256 pixels, providing clarity and consistency for model training.

File Format: JPG Color Space: RGB

Intended Applications
This dataset is primed for use in developing and testing AI models specialized in multi-class animal recognition. It offers valuable resources for researchers and hobbyists in fields such as zoology, pet technology, and biodiversity conservation.

Acknowledgements and Licensing
This dataset is a collective effort of various photographers and organizations. All images are distributed with permissions for academic and non-commercial usage, provided that proper attribution is given to the original sources.

# Про набір даних

***Короткий опис набору даних***

Набір даних «Класифікація зображень тварин» - це повна колекція зображень, призначена для розробки та оцінки моделей машинного навчання в галузі комп'ютерного зору. Він містить 3 000 зображень у форматі JPG, ретельно сегментованих на три класи, що представляють найпоширеніших домашніх та диких тварин: котів, собак та змій.

Зміст набору даних
cats/: Набір з 1,000 JPG зображень котів, що демонструють широкий спектр порід, середовищ та поз.

собаки/: Різноманітна добірка з 1,000 зображень собак, що демонструє безліч порід у різних видах діяльності та середовищах.

змії/: Підбірка з 1 000 зображень змій, на яких зображені численні види як у природному, так і в контрольованому середовищі існування. Деталі зображення:

Роздільна здатність: Кожне зображення має однакову роздільну здатність 256x256 пікселів, що забезпечує чіткість і послідовність для навчання моделі.

Формат файлу: JPG Колірний простір: RGB

Призначення
Цей набір даних призначений для використання при розробці та тестуванні моделей штучного інтелекту, що спеціалізуються на розпізнаванні тварин різних класів. Він пропонує цінні ресурси для дослідників і аматорів у таких галузях, як зоологія, технології для домашніх тварин і збереження біорізноманіття.

Подяки та ліцензування
Цей набір даних є результатом колективних зусиль різних фотографів та організацій. Всі зображення розповсюджуються з дозволом на академічне та некомерційне використання за умови належного посил

In [18]:
import os
import pandas as pd
import numpy as np

from PIL import Image
from PIL import ImageMath

In [20]:
files_image_path = '../../classification/animal/data/'
# Список категорій (назви папок)
categories = ["cats", "dogs", "snakes"]

# Створюємо список для зберігання даних
image_list = []
labels = []

In [21]:
# Читаємо зображення з кожної категорії
for category in categories:
    folder_path = os.path.join(files_image_path, category)
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg') or f.endswith('.png')]
    
    for image_file in image_files:
        img = Image.open(os.path.join(folder_path, image_file))
        img_array = np.array(img)  # Перетворюємо у масив
        image_list.append(img_array)
        labels.append(category)  # Додаємо мітку (категорію)

# Створюємо DataFrame
df = pd.DataFrame({"image": image_list, "label": labels})
print(df.head())  # Перевіряємо структуру

                                               image label
0  [[[64, 66, 87], [68, 70, 91], [73, 75, 96], [7...  cats
1  [[[40, 61, 54], [41, 62, 55], [43, 64, 57], [4...  cats
2  [[[94, 93, 91], [96, 95, 93], [99, 98, 96], [1...  cats
3  [[[189, 187, 190], [190, 188, 191], [190, 188,...  cats
4  [[[203, 186, 156], [207, 190, 160], [206, 189,...  cats


In [22]:
df.columns

Index(['image', 'label'], dtype='object')